In [1]:
import os
import sys

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import pandas as pd

from ray import tune
from ray.air import session
from ray.tune.schedulers import ASHAScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pathlib import Path

from src.models import *
from src.data import Data
from src.train import fit

In [2]:
target_variable = 'Flow_Kalltveit'
file_name = "cleaned_data_1.csv"
data_dir = "../data"
datetime_variable = "Datetime"

In [3]:
def train_model(config, checkpoint_dir=None):
    use_GPU = torch.cuda.is_available()
    if use_GPU:
        mode = {"name": "cuda", "device": torch.device("cuda")}
    else:
        mode = {"name": "cpu", "device": torch.device("cpu")}

    # Define hyperparameters
    train_size = 0.7
    val_size = 0.2
    test_size = 0.1

    sequence_length = config['sequence_length']
    batch_size = config['batch_size']
    num_epochs = config['num_epochs']
    lr = config['learning_rate']
    weight_decay = config['weigth_decay']

    # Set data file
    data_file = config['data_file']
    datetime_variable = config['datetime']

    data = Data(data_file, datetime_variable)

    # Select variables to use
    vars = config['variables']
    target_variable = config['target_variable']
    X, y = data.data_transformation(sequence_length=sequence_length, target_variable=target_variable, columns_to_transformation=vars)

    # Split the data
    X_train, y_train, X_val, y_val, X_test, y_test = data.split_data(X, y, train_size=train_size, val_size=val_size, test_size=test_size)
    train_dataloader = data.create_dataloader(X_train, y_train, sequence_length, batch_size=batch_size, shuffle=True)
    val_dataloader = data.create_dataloader(X_val, y_val, sequence_length, batch_size=batch_size, shuffle=False)
    test_dataloader = data.create_dataloader(X_test, y_test, sequence_length, batch_size=batch_size, shuffle=False)

    # Model inputs
    if vars:
        input_size = len(vars) + 1
    else:
        input_size = 1
    hidden_size = config['hidden_size']
    num_layers = config['num_layers']
    output_size = 1

    if config['arch'] == "FCN":
        net = FCN(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] ==  "FCNTemporalAttention":
        net = FCNTemporalAttention(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] == "LSTM":
        net = LSTM(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] == "LSTMTemporalAttention":
        net = LSTMTemporalAttention(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] == "LSTMSpatialAttention":
        net = LSTMSpatialAttention(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] == "LSTMSpatialTemporalAttention":
        net = LSTMSpatialTemporalAttention(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )

    data_loader = {
    "train": train_dataloader,
    "val": val_dataloader,
    "test": test_dataloader,
    }
    
    net.to(mode["device"])

    loss_function = nn.MSELoss().to(mode["device"])
    optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    fit(net, loss_function, optimizer, data_loader, num_epochs, mode, checkpoint_dir, use_amp=True)

In [4]:
# Because I do report validation loss first
os.environ["TUNE_DISABLE_STRICT_METRIC_CHECKING"] = "1"

In [5]:
from functools import partial
from ray.tune.schedulers import PopulationBasedTraining


config = {
    "data_file": file_name,
    "datetime":  datetime_variable,
    "target_variable": target_variable,
    "arch": tune.grid_search(["LSTM"]), # "FCN", "FCNTemporalAttention", "LSTMTemporalAttention", "LSTM", "LSTMSpatialAttention", "LSTMSpatialTemporalAttention"
    "sequence_length": tune.choice([25]),
    'num_epochs': tune.choice([30]),
    'num_layers': tune.choice([2, 3, 4]),
    "learning_rate": tune.loguniform(1e-4, 1e-1),
    "weigth_decay": tune.choice([0, 0.001, 0.0001]),
    "batch_size": tune.choice([256, 512]),
    "hidden_size": tune.choice([32, 64]),
    "variables": tune.grid_search([
        None,
    ])
}


"""scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    metric="val_loss",
    mode="min",
    perturbation_interval=2,
    # Because I do report validation loss first
    require_attrs=False,
    hyperparam_mutations={
        "weigth_decay": tune.uniform(0.0, 0.3),
        "learning_rate": tune.uniform(1e-5, 5e-5),
    },
    )"""


scheduler = ASHAScheduler(
    metric="val_loss",
    mode="min",
    max_t=100,
    grace_period=5,
    reduction_factor=2
)

reporter = tune.JupyterNotebookReporter(
        parameter_columns={
            "weight_decay": "w_decay",
            "learning_rate": "lr",
            "num_epochs": "num_epochs"
        },
        metric_columns=[
            "train_loss", "val_loss", "test_loss", "epoch", "training_iteration"
        ])

analysis = tune.run(
    train_model, # TODO: partial(train_cifar, data_dir=data_dir),
    resources_per_trial={"cpu": 12, "gpu": 1},
    config=config,
    num_samples=1,
    scheduler=scheduler,
    progress_reporter=reporter,
    name="inflow_forecasting",
    verbose=True
)

2023-04-02 18:02:58,472	INFO worker.py:1553 -- Started a local Ray instance.
c:\Code\Master\hydro-ml\.venv\lib\site-packages\ray\tune\trainable\function_trainable.py:612: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/master/tune/api_docs/trainable.html

  warnings.warn(
 60%|█████▉    | 53/89 [00:00<00:00, 169.11it/s]
2023-04-02 18:03:27,342	INFO tune.py:798 -- Total run time: 27.47 seconds (27.42 seconds for the tuning loop).


In [6]:
print("Best config: ", analysis.get_best_config(
   metric="val_loss", mode="min"))
# Get a dataframe for analyzing trial results.
df = analysis.results_df

Best config:  {'data_file': 'cleaned_data_1.csv', 'datetime': 'Datetime', 'target_variable': 'Flow_Kalltveit', 'arch': 'LSTM', 'sequence_length': 25, 'num_epochs': 30, 'num_layers': 4, 'learning_rate': 0.00017598044440355847, 'weigth_decay': 0, 'batch_size': 512, 'hidden_size': 32, 'variables': None}


100%|██████████| 13/13 [00:00<00:00, 267.77it/s]


In [7]:
df

,test_loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_total_s,...,config/target_variable,config/arch,config/sequence_length,config/num_epochs,config/num_layers,config/learning_rate,config/weigth_decay,config/batch_size,config/hidden_size,config/variables
trial_id,,,,,,,,,,,,,,,,,,,,,
d71a9_00000,22.866741,0.051553,True,None,None,61,551d462ccca3435e9d1a005c4803b710,2023-04-02_18-03-27,1680451407,23.883509,...,Flow_Kalltveit,LSTM,25,30,4,0.000176,0,512,32,None


In [8]:
"""
    "model": {
        "hidden_size": tune.choice([32, 64, 128]),
        "num_layers": tune.choice([1, 2, 3]),
    },

    model = YourModel(**config["model"]).to(device)
"""

'\n    "model": {\n        "hidden_size": tune.choice([32, 64, 128]),\n        "num_layers": tune.choice([1, 2, 3]),\n    },\n\n    model = YourModel(**config["model"]).to(device)\n'